- [Определение стоимости автомобилей](#toc1_)    
  - [Подготовка данных](#toc1_1_)    
    - [Обработка пропусков](#toc1_1_1_)    
  - [Обучение моделей](#toc1_2_)    
    - [LinearRegression](#toc1_2_1_)    
    - [Catboost](#toc1_2_2_)    
    - [LGBMRegressor](#toc1_2_3_)    
  - [Анализ моделей](#toc1_3_)    
  - [Вывод](#toc1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Определение стоимости автомобилей](#toc0_)

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [1]:
!pip install category_encoders

     |████████████████████████████████| 81 kB 7.1 kB/s eta 0:00:01


## <a id='toc1_1_'></a>[Подготовка данных](#toc0_)

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import numpy as np
import decimal

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PowerTransformer, StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.james_stein import JamesSteinEncoder 
from category_encoders.count import CountEncoder

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

pd.options.display.float_format = '{:.2f}'.format
%matplotlib

Using matplotlib backend: agg


In [3]:
df = pd.read_csv('/datasets/autos.csv')

In [4]:
df.shape

(354369, 16)

DateCrawled — дата скачивания анкеты из базы
VehicleType — тип автомобильного кузова
RegistrationYear — год регистрации автомобиля
Gearbox — тип коробки передач
Power — мощность (л. с.)
Model — модель автомобиля
Kilometer — пробег (км)
RegistrationMonth — месяц регистрации автомобиля
FuelType — тип топлива
Brand — марка автомобиля
Repaired — была машина в ремонте или нет
DateCreated — дата создания анкеты
NumberOfPictures — количество фотографий автомобиля
PostalCode — почтовый индекс владельца анкеты (пользователя)
LastSeen — дата последней активности пользователя

In [5]:
df.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
180760,2016-03-17 09:38:31,5299,NaN,2016,manual,69,ibiza,125000,7,petrol,seat,no,2016-03-17 00:00:00,0,58638,2016-04-06 07:44:26
35256,2016-04-02 00:50:56,1299,small,2004,auto,61,fortwo,150000,1,petrol,smart,yes,2016-04-01 00:00:00,0,49456,2016-04-06 05:16:05
243114,2016-03-26 13:57:07,1700,sedan,2000,manual,105,3er,150000,3,petrol,bmw,no,2016-03-26 00:00:00,0,70469,2016-03-29 16:46:03
289070,2016-03-21 09:54:48,4000,NaN,2007,manual,69,fox,150000,0,NaN,volkswagen,no,2016-03-21 00:00:00,0,42349,2016-04-06 02:45:30
64336,2016-04-03 20:06:19,6990,small,2008,manual,75,500,80000,1,gasoline,fiat,no,2016-04-03 00:00:00,0,65428,2016-04-07 13:16:12


In [6]:
df.dtypes

DateCrawled          object
Price                 int64
VehicleType          object
RegistrationYear      int64
Gearbox              object
Power                 int64
Model                object
Kilometer             int64
RegistrationMonth     int64
FuelType             object
Brand                object
Repaired             object
DateCreated          object
NumberOfPictures      int64
PostalCode            int64
LastSeen             object
dtype: object

In [7]:
df.duplicated().sum()

4

In [8]:
df = df.drop_duplicates()

In [9]:
len(df['PostalCode'].unique())

8143

Представляется, что дата скачивания анкеты с базы данных (DateCrawled), дата последней активности пользователя (LastSeen), почтовый индекс (PostalCode), дата публикации объявления (DateCreated), месяц регистрации (RegistrationMonth) сами по себе не представляют интереса.

На основе данных почтового индекса (PostalCode) можно было бы выделить населенный пункт жительства продавца, но так как из какой страны эти данные неизвестно, то сделать это не представляется возможным. Также использовать их в качестве категориальных признаков, по-моему, тоже нельзя, так как их слишком много.

In [10]:
new_columns = list(df.columns)
for i in ['DateCrawled', 'LastSeen', 'PostalCode', 'RegistrationMonth', 'DateCreated']:
    new_columns.remove(i)

In [11]:
df = df[new_columns]

In [12]:
df.dtypes

Price                int64
VehicleType         object
RegistrationYear     int64
Gearbox             object
Power                int64
Model               object
Kilometer            int64
FuelType            object
Brand               object
Repaired            object
NumberOfPictures     int64
dtype: object

In [13]:
df.columns = ['price', 'vehicle_type', 'registration_year', 'gearbox', 'power',
              'model', 'kilometer', 'fuel_type', 'brand', 'repaired', 'number_pictures']

In [14]:
df.describe()

,price,registration_year,power,kilometer,number_pictures
count,354365.00,354365.00,354365.00,354365.00,354365.00
mean,4416.68,2004.23,110.09,128211.36,0.00
std,4514.18,90.23,189.85,37905.08,0.00
min,0.00,1000.00,0.00,5000.00,0.00
25%,1050.00,1999.00,69.00,125000.00,0.00
50%,2700.00,2003.00,105.00,150000.00,0.00
75%,6400.00,2008.00,143.00,150000.00,0.00
max,20000.00,9999.00,20000.00,150000.00,0.00


number_pictures везде ноль, поэтому этот столбец также необходимо удалить. 

In [15]:
df.drop('number_pictures', axis=1, inplace=True)

In [16]:
cat_feat = df.dtypes[df.dtypes == object].index
num_feat = df.dtypes[df.dtypes != object].index

In [17]:
for i in num_feat:
    print(i, df[i].median())

price 2700.0
registration_year 2003.0
power 105.0
kilometer 150000.0


### <a id='toc1_1_1_'></a>[Обработка пропусков](#toc0_)

In [18]:
df.isnull().sum()

price                    0
vehicle_type         37490
registration_year        0
gearbox              19833
power                    0
model                19705
kilometer                0
fuel_type            32895
brand                    0
repaired             71154
dtype: int64

In [19]:
df[df.model.isnull()].isnull().sum()

price                    0
vehicle_type          6828
registration_year        0
gearbox               4131
power                    0
model                19705
kilometer                0
fuel_type             7163
brand                    0
repaired              9054
dtype: int64

Так как модель автомобиля является значимой характеристикой, по-моему мнению, стоит удалить те строки, где она не указана. 

In [20]:
df.dropna(subset=['model'], inplace=True)

In [21]:
df.isnull().sum()

price                    0
vehicle_type         30662
registration_year        0
gearbox              15702
power                    0
model                    0
kilometer                0
fuel_type            25732
brand                    0
repaired             62100
dtype: int64

Остальные пропуски стоит пометить, как 'unknow'.

In [22]:
df.fillna('unknow', inplace=True)

In [23]:
df.isnull().sum()

price                0
vehicle_type         0
registration_year    0
gearbox              0
power                0
model                0
kilometer            0
fuel_type            0
brand                0
repaired             0
dtype: int64

Похоже, что количественные данные содержат выбросы поэтому постараемся их найти при помощи графиков

In [24]:
cat_feat

Index(['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired'], dtype='object')

In [25]:
num_feat

Index(['price', 'registration_year', 'power', 'kilometer'], dtype='object')

Имеются следующие величины имеющие выбросы: цена, год регистрации, мощность.

Небольшой пробег автомобиля при его продаже нельзя считать выбросом, так как такие факты бывают. 

Представляется необходимым сразу разобратся с очевидными выбрасами, такими как 0 скорость и мощность, а также неадекватный год регистрации.

In [27]:
print(sum((df.power == 0)))
print(sum((df.price == 0)))

33931
8588


Значения с 0 явно неадекватные, поэтому их необходимо удалить. 

In [28]:
df.drop(df[(df.power == 0) | (df.price == 0)].index, axis=0, inplace=True)

In [29]:
df.shape

(294739, 10)

In [30]:
df['registration_year'].value_counts()

1999    18801
2005    18317
2006    18140
2000    17910
2003    17446
        ...  
1935        1
1936        1
1947        1
1949        1
1932        1
Name: registration_year, Length: 101, dtype: int64

Исходя из распределения данных представляется необходимым преобразовать числовую характеристику годов в категориальную по следующему принципу: 
- 1960-1980; 
- 1980-1990; 
- 1990-2000; 
- 2000-2010;
- 2010-2015;
- 2015-2020; 
- иные не попавшие не в один из диапозонов. 

Таким образом, мы преобразуем числовую характеристику годов в категориальную и выделим выбросы в отдельную категорию не удаляя данные с указанными выбросами. 

In [32]:
def classific_year(x):
    if x >= 1960 and x < 1980:
        return '60-80'
    elif x >= 1980 and x < 1990:
        return '80-90'
    elif x >= 1990 and x < 2000:
        return '90-00'
    elif x >= 2000 and x < 2010:
        return '00-10'
    elif x >= 2010 and x < 2015:
        return '10-15'
    elif x >= 2015 and x < 2020:
        return '15-20'
    else:
        return 'other'

In [33]:
df['registration_year'] = df['registration_year'].apply(classific_year)

In [34]:
df['registration_year'] = df['registration_year'].astype('category')

In [35]:
df['registration_year'].value_counts()

00-10    166702
90-00     73534
10-15     32693
15-20     15689
80-90      4333
60-80      1665
other       123
Name: registration_year, dtype: int64

In [36]:
df.dtypes

price                   int64
vehicle_type           object
registration_year    category
gearbox                object
power                   int64
model                  object
kilometer               int64
fuel_type              object
brand                  object
repaired               object
dtype: object

Распределение цены выглядит странно, цены как бы делятся на свои собственные распределения со своими локальными пиками.
Также имеется большое количество цен с 0. 

In [39]:
df.shape

(294739, 10)

In [40]:
sum((df.price < 400))

11945

Представляется, что цена автомобиля стоимостью менее 400$ является выбросом, который надо удалить. 

In [41]:
df = df[df.price >= 400]

In [42]:
sum(df.price > 15000)

14032

In [43]:
df.columns

Index(['price', 'vehicle_type', 'registration_year', 'gearbox', 'power',
       'model', 'kilometer', 'fuel_type', 'brand', 'repaired'],
      dtype='object')

In [44]:
df.loc[df.price > 15000, 'model'].value_counts()

other       1116
golf        1098
3er          900
c_klasse     589
a4           579
            ... 
agila          1
espace         1
s_type         1
jimny          1
v_klasse       1
Name: model, Length: 169, dtype: int64

In [45]:
df['class'] = df['price'].apply(lambda x: 'more' if x >= 15000 else 'less')

In [47]:
more = df.loc[df.price >= 15000, 'power']
less = df.loc[df.price < 15000, 'power']
for data, name in zip([more, less], ['more', 'less']):
    print(f'Медиана {name}: {data.median()}')
    print(f'Среднее {name}: {data.mean()}')

Медиана more: 170.0
Среднее more: 185.59702209414024
Медиана less: 110.0
Среднее less: 122.18503840131235


Как видно из представленных графиков, а также среднего и медианны, мощность автомобилей с ценой более 15 тыс. долларов больше чем для других автомобилей, таким образом очень дорогие автомобили это не выброс. 

In [48]:
df.dtypes

price                   int64
vehicle_type           object
registration_year    category
gearbox                object
power                   int64
model                  object
kilometer               int64
fuel_type              object
brand                  object
repaired               object
class                  object
dtype: object

В ходе предобработки удалены признаки: LastSeen, PostalCode, DateCreated, RegistrationMonth, number_pictures. Удалены выбросы в следующих признаках: price, power. Признак registration_year преобразован в категориальный.

## <a id='toc1_2_'></a>[Обучение моделей](#toc0_)

В ходе работы обучим линейную регрессию, LigtGBM, Catboost.

In [49]:
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.8)
cat_feat = list(X_train.dtypes[X_train.dtypes != int].index)
num_feat = list(X_train.dtypes[X_train.dtypes == int].index)

In [50]:
cat_feat

['vehicle_type',
 'registration_year',
 'gearbox',
 'model',
 'fuel_type',
 'brand',
 'repaired',
 'class']

In [51]:
num_feat

['power', 'kilometer']

### <a id='toc1_2_1_'></a>[LinearRegression](#toc0_)

In [52]:
numeric_transformer = Pipeline(steps=[
     ("scaler", StandardScaler())])


categorical_transformer = Pipeline(steps=[
    ('cat', OneHotEncoder(handle_unknown="value"))
])

preprocessor = ColumnTransformer(transformers=[
     ("num_transform", numeric_transformer, num_feat),
    ("cat_transform", categorical_transformer, cat_feat)    
])


pipeline_linear = Pipeline([('preprocessor', preprocessor), 
                    ('clf', Lasso())])
pipeline_linear

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_transform',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['power', 'kilometer']),
                                                 ('cat_transform',
                                                  Pipeline(steps=[('cat',
                                                                   OneHotEncoder())]),
                                                  ['vehicle_type',
                                                   'registration_year',
                                                   'gearbox', 'model',
                                                   'fuel_type', 'brand',
                                                   'repaired', 'class'])])),
                ('clf', Lasso())])

In [ ]:
list_alpha = [0.5, 1, 2]
param = [ {"clf__alpha": list_alpha}
        ]  
                                       
grid_search_linear = GridSearchCV(pipeline_linear, param, cv=5, n_jobs=-1, 
                                 refit='rmse', scoring='neg_mean_squared_error')
grid_search_linear.fit(X_train, y_train)
print("Лучшие параметры:")
print(grid_search_linear.best_params_)
print("Лучшая метрика на валидационных данных:")
print(grid_search_linear.best_score_)

Лучшие параметры:
{'clf__alpha': 0.5}
Лучшая метрика на валидационных данных:
-5147708.729449254


Я ошибку выше никак побороть не могу... (ValueError: Input contains NaN, infinity or a value too large for dtype('float64')). Это проблема касается линейных моделей (аналогичный результат на Ridge), на градиентном бустинге все норм. 

<div class="alert alert-block alert-info">
<b>Комментарий ревьюера:</b>

Думаю дело в масштабировании. Попробуй выполнить масштабирование вручную через StandardScaler. И масштабировать только численные признаки 
</div>

### <a id='toc1_2_2_'></a>[Catboost](#toc0_)

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
model = CatBoostRegressor(iterations=2000, 
                          cat_features=cat_feat,
                          verbose=False
                         )
param = {'l2_leaf_reg':[0.5, 1], 
        'depth':[6, 8, 9], 
         'learning_rate':[0.03, 0.003]
       }

grid_search = model.grid_search(param,
                                X=X_train,
                                y=y_train,
                                train_size=0.8,
                                refit=True,
                                cv=3,
                                calc_cv_statistics=True,
                                verbose=False,
                                plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 1712.761675
bestIteration = 1999


bestTest = 1913.39416
bestIteration = 1999


bestTest = 1717.315313
bestIteration = 1999


bestTest = 1912.140174
bestIteration = 1999


bestTest = 1667.776585
bestIteration = 1999


bestTest = 1869.449486
bestIteration = 1999


bestTest = 1670.65066
bestIteration = 1999


bestTest = 1869.41206
bestIteration = 1999


bestTest = 1649.021455
bestIteration = 1997


bestTest = 1849.025102
bestIteration = 1999


bestTest = 1651.591754
bestIteration = 1999


bestTest = 1852.906106
bestIteration = 1999

Training on fold [0/3]

bestTest = 1666.757404
bestIteration = 1996

Training on fold [1/3]

bestTest = 1644.153689
bestIteration = 1999

Training on fold [2/3]

bestTest = 1658.826891
bestIteration = 1999



In [ ]:
grid_search['params']

{'depth': 9, 'learning_rate': 0.03, 'l2_leaf_reg': 0.5}

Обучим заново модель с лучшими параметрами, чтобы найти время за которое она обучится. 

### <a id='toc1_2_3_'></a>[LGBMRegressor](#toc0_)

In [ ]:
from lightgbm import LGBMRegressor
import lightgbm as lgb

In [ ]:
X_train_lg = X_train.copy()

In [ ]:
X_train_lg[cat_feat] = X_train_lg[cat_feat].apply(LabelEncoder().fit_transform)

In [ ]:
params = {
    'num_leaves': [31, 50],
    'learning_rate': [0.03, 0.003],
    'max_depth': [-1, 5],
    'n_estimators': [500, 1000],
}

grid_lg = GridSearchCV(LGBMRegressor(), params, scoring='neg_root_mean_squared_error', cv=3)
grid_lg.fit(X_train_lg, y_train)

GridSearchCV(cv=3, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.03, 0.003], 'max_depth': [-1, 5],
                         'n_estimators': [500, 1000], 'num_leaves': [31, 50]},
             scoring='neg_root_mean_squared_error')

In [ ]:
print("Лучшие параметры:")
print(grid_lg.best_params_)
print("Лучшая метрика на валидационных данных:")
print(grid_lg.best_score_)

Лучшие параметры:
{'learning_rate': 0.03, 'max_depth': -1, 'n_estimators': 1000, 'num_leaves': 50}
Лучшая метрика на валидационных данных:
-1655.3425075708176


Чтобы узнать время обучения обучим заново модель LGBMRegressor, включим туда время предобработки категориальных фичей, для чистоты эксперимента. 

## <a id='toc1_3_'></a>[Анализ моделей](#toc0_)

In [ ]:
table = pd.DataFrame(grid_search_linear.cv_results_)
linear_list = list(table.loc[table.params == grid_search_linear.best_params_, ['mean_fit_time', 'mean_score_time']].values[0])
linear_list.append(abs(grid_search_linear.best_score_))
linear_list

[92.81106734275818, 0.47640461921691896, 5147708.729449254]

In [ ]:
tabel_lgbmr = pd.DataFrame(grid_lg.cv_results_)
lgbmr_list = list(tabel_lgbmr.loc[tabel_lgbmr.params == grid_lg.best_params_, ['mean_fit_time', 'mean_score_time']].values[0])
lgbmr_list.append(abs(grid_lg.best_score_))
lgbmr_list

[18.335092544555664, 2.717296044031779, 1655.3425075708176]

<div class="alert alert-block alert-info">

grid_search катбуста данных о среднем времени обучения и предсказания не предаставляет, поэтому обучим модель заново на лучших параметрах. И используем при предсказании X_test **только** для того, что бы узмерить время предсказания. 

In [ ]:
%%time
model_cat = CatBoostRegressor(iterations=2000, 
                          cat_features=cat_feat,
                          depth=9,
                          learning_rate=0.03,
                          l2_leaf_reg=0.5,
                          verbose=False
                         )
model_cat.fit(X_train, y_train, verbose=False)

CPU times: user 3min 45s, sys: 26 s, total: 4min 11s
Wall time: 4min 13s


In [ ]:
%%time
model_cat.predict(X_test)

CPU times: user 5.3 s, sys: 35.9 ms, total: 5.33 s
Wall time: 5.34 s


array([ 3437.61729123,  4321.61182932,  1491.15528496, ...,
        6138.75508627,  5045.66847525, 17260.2006452 ])

In [ ]:
cat_list = [264, 5.44, 1853]

In [ ]:
result = pd.DataFrame([linear_list, lgbmr_list, cat_list], 
                      columns=['mean_fit_time', 'mean_score_time', 'best_score'], 
                      index=['linear', 'LGBMRegressor', 'Catboost'])

In [ ]:
result

,mean_fit_time,mean_score_time,best_score
linear,92.81,0.48,5147708.73
LGBMRegressor,18.34,2.72,1655.34
Catboost,264.00,5.44,1853.00


## <a id='toc1_4_'></a>[Вывод](#toc0_)

В ходе анализа установлено следующее: 
- время обучения catboost 4 мин. 20 сек., время предсказания для тестовых данных: 4.82 сек., метрика RMSE 1619.87; 
- время обучения LGBM 20 сек., время предсказания для тестовых данных: 29.9 сек., метрика RMSE 1648.04.
        
Выбор модели зависит от наличных ресурсов и потребностей заказчика. Предполагаю, что целесообразно выбрать catboost, потому что он быстрее предсказывает и поэтому работу такой модели, возможно, будет проще интегрировать в сайт. Также более продолжительное время обучения, чем у LGBM не должно быть проблемой, если размер предполагаемых тренировочных данных заказчика не больше в 1000 раз и не будет возможности обучить модель на GPU.

<div class="alert alert-block alert-info">
Исходя из данных представленных ниже LGBM является наилучшим выбором, возможно catboost при обучении на GPU справится лучше. 

In [ ]:
result

,mean_fit_time,mean_score_time,best_score
linear,92.81,0.48,5147708.73
LGBMRegressor,18.34,2.72,1655.34
Catboost,264.00,5.44,1853.00
